# Simple Football Arbitrage Bot

## Brief rundown on how this works:

Different sports bookmakers have different odds on the outcomes of a match (Team A wins, Draw, Team B wins). 

The odds of these match outcomes add up to exactly 100%, but a bookmaker will shift the odds such that the odds of all match otucomes is above 100%, the difference is the bookmaker's edge (i.e. profit). 

For example: PaddyPower rates the odds of Barnsley vs Port Vale (on 2023-07-23) to be 1.62:5:3.5 (win1:win2:draw). Since probabilities are the inverse of the odds we have 0.617:0.20:0.285, this adds up to 1.102, so PaddyPower's edge is 10.2%.

For us to make profit, we have to find odds from different bookmakers that total under 100%. But in reality, since all three outcomes total a 100% probability, we will make money off this difference.

I will be using the-odds-api.com's free API to collect odds from different bookmakers, and then do the rest on here.

In [37]:
import requests
import pandas as pd
from pandas import json_normalize

In [9]:
with open('API_KEY.txt') as f:
    API_KEY = f.read()
print(f"API key is {API_KEY}")

API key is f6198e12290f5456ce6161b2a5d38720


The next block is straight from the-odds-api.com's sample code. No need to make it myself.

In [13]:
SPORT = 'soccer_england_league1' # use the sport_key from the /sports endpoint below, or use 'upcoming' to see the next 8 games across all sports

REGIONS = 'uk' # uk | us | eu | au. Multiple can be specified if comma delimited

MARKETS = 'h2h' # h2h | spreads | totals. Multiple can be specified if comma delimited

ODDS_FORMAT = 'decimal' # decimal | american

DATE_FORMAT = 'iso' # iso | unix


# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
#
# Now get a list of live & upcoming games for the sport you want, along with odds for different bookmakers
# This will deduct from the usage quota
# The usage quota cost = [number of markets specified] x [number of regions specified]
# For examples of usage quota costs, see https://the-odds-api.com/liveapi/guides/v4/#usage-quota-costs
#
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

odds_response = requests.get(
    f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds',
    params={
        'api_key': API_KEY,
        'regions': REGIONS,
        'markets': MARKETS,
        'oddsFormat': ODDS_FORMAT,
        'dateFormat': DATE_FORMAT,
    }
)

if odds_response.status_code != 200:
    print(f'Failed to get odds: status_code {odds_response.status_code}, response body {odds_response.text}')

else:
    odds_json = odds_response.json()
    print('Number of events:', len(odds_json))
    print(odds_json)

    # Check the usage quota
    print('Remaining requests', odds_response.headers['x-requests-remaining'])
    print('Used requests', odds_response.headers['x-requests-used'])

Number of events: 12
[{'id': '8372d6cc49af461e9a719f8b63f74055', 'sport_key': 'soccer_england_league1', 'sport_title': 'League 1', 'commence_time': '2023-08-05T14:00:00Z', 'home_team': 'Barnsley', 'away_team': 'Port Vale', 'bookmakers': [{'key': 'boylesports', 'title': 'BoyleSports', 'last_update': '2023-07-23T00:42:27Z', 'markets': [{'key': 'h2h', 'last_update': '2023-07-23T00:42:27Z', 'outcomes': [{'name': 'Barnsley', 'price': 1.7}, {'name': 'Port Vale', 'price': 4.75}, {'name': 'Draw', 'price': 3.3}]}]}, {'key': 'paddypower', 'title': 'Paddy Power', 'last_update': '2023-07-23T00:41:25Z', 'markets': [{'key': 'h2h', 'last_update': '2023-07-23T00:41:25Z', 'outcomes': [{'name': 'Barnsley', 'price': 1.62}, {'name': 'Port Vale', 'price': 5.0}, {'name': 'Draw', 'price': 3.5}]}]}, {'key': 'skybet', 'title': 'Sky Bet', 'last_update': '2023-07-23T00:42:11Z', 'markets': [{'key': 'h2h', 'last_update': '2023-07-23T00:42:11Z', 'outcomes': [{'name': 'Barnsley', 'price': 1.7}, {'name': 'Port Vale',

Now we'll make odds_json more readable using pandas.

First we have to make a function that reads odds_json into something that pandas can handle.

In [69]:
def extract_info(item):
    """
    Extracts match and bookmaker information from a given dictionary representing a single match.

    Parameters:
        item: A dictionary containing information about a specific match.
        
        The dictionary will contain the following keys:
                     - 'home_team': The name of the home team.
                     - 'away_team': The name of the away team.
                     - 'commence_time': The date and time when the match starts (in ISO 8601 format).
                     - 'bookmakers': A list of dictionaries, each representing a bookmaker offering odds for the match.
                       Each bookmaker dictionary should have the following keys:
                       - 'title': The name of the bookmaker.
                       - 'last_update': The date and time when the bookmaker's odds were last updated (in ISO 8601 format).
                       - 'markets': A list of dictionaries, each representing a market for the match (e.g., 'h2h' for
                         head-to-head market).
                         Each market dictionary should have the following keys:
                         - 'outcomes': A list of dictionaries, each representing an outcome in the market.
                           Each outcome dictionary should have the following keys:
                           - 'name': The name of the outcome (e.g., team name or 'Draw' for a draw).
                           - 'price': The odds value for the corresponding outcome.

    Returns:
        extracted_data: A list of dictionaries, each with information for a specific bookmaker and market outcome.
              Each dictionary has the following keys:
              - 'team 1 name': The name of the home team.
              - 'team 2 name': The name of the away team.
              - 'date': The date and time when the match starts (in ISO 8601 format).
              - 'bookmaker name': The name of the bookmaker.
              - 'last bookmaker update': The date and time when the bookmaker's odds were last updated (in ISO 8601 format).
              - 'team 1 odds': The odds value for the home team.
              - 'team 2 odds': The odds value for the away team.
              - 'Draw odds': The odds value for a draw outcome.
    """
    extracted_data = []
    for bookmaker in item['bookmakers']: # We iterate over bookmakers and markets because they are nested in the json
        for market in bookmaker['markets']:
            outcomes = {outcome['name']: outcome['price'] for outcome in market['outcomes']}
            extracted_data.append({
                'Home Team': item['home_team'],
                'Away Team': item['away_team'],
                'date': item['commence_time'],
                'Bookmaker Name': bookmaker['title'],
                'Last bookmaker update': bookmaker['last_update'],
                'team 1 win odds': outcomes.get(item['home_team'], None),
                'team 2 win odds': outcomes.get(item['away_team'], None),
                'Draw odds': outcomes.get('Draw', None),
            })
    return extracted_data


In [73]:
data = odds_json

match_tables = {} # Dictionary that stores the DataFrames for every match

for item in data:
    match_name = f"{item['home_team']} vs {item['away_team']}"
    match_data = extract_info(item) 
    if match_name not in match_tables:
        match_tables[match_name] = pd.DataFrame(match_data)
    else:
        match_tables[match_name] = pd.concat([match_tables[match_name], pd.DataFrame(match_data)])
        
# if multiple occurrences of the same match are encountered, their data is concatenated

# Display each match's DataFrame
for match_name, match_df in match_tables.items():
    print(f"Table for {match_name}:")
    display(match_df)
    print("\n")

Table for Barnsley vs Port Vale:


,Home Team,Away Team,date,Bookmaker Name,Last bookmaker update,team 1 win odds,team 2 win odds,Draw odds
0,Barnsley,Port Vale,2023-08-05T14:00:00Z,BoyleSports,2023-07-23T00:42:27Z,1.70,4.75,3.30
1,Barnsley,Port Vale,2023-08-05T14:00:00Z,Paddy Power,2023-07-23T00:41:25Z,1.62,5.00,3.50
2,Barnsley,Port Vale,2023-08-05T14:00:00Z,Sky Bet,2023-07-23T00:42:11Z,1.70,4.50,3.50
3,Barnsley,Port Vale,2023-08-05T14:00:00Z,Bet Victor,2023-07-23T00:41:48Z,1.70,4.33,3.50
4,Barnsley,Port Vale,2023-08-05T14:00:00Z,Coral,2023-07-23T00:42:19Z,1.80,4.60,3.40
5,Barnsley,Port Vale,2023-08-05T14:00:00Z,Ladbrokes,2023-07-23T00:42:02Z,1.80,4.60,3.40
6,Barnsley,Port Vale,2023-08-05T14:00:00Z,888sport,2023-07-23T00:41:07Z,1.73,4.50,3.40
7,Barnsley,Port Vale,2023-08-05T14:00:00Z,Betway,2023-07-23T00:41:28Z,1.70,4.20,3.40
8,Barnsley,Port Vale,2023-08-05T14:00:00Z,Betfair,2023-07-23T00:40:35Z,1.68,4.40,3.20
9,Barnsley,Port Vale,2023-08-05T14:00:00Z,Betfair,2023-07-23T00:40:35Z,2.04,6.40,4.40




Table for Blackpool vs Burton Albion:


,Home Team,Away Team,date,Bookmaker Name,Last bookmaker update,team 1 win odds,team 2 win odds,Draw odds
0,Blackpool,Burton Albion,2023-08-05T14:00:00Z,BoyleSports,2023-07-23T00:42:27Z,1.55,5.00,3.80
1,Blackpool,Burton Albion,2023-08-05T14:00:00Z,Paddy Power,2023-07-23T00:41:25Z,1.53,5.00,4.00
2,Blackpool,Burton Albion,2023-08-05T14:00:00Z,Sky Bet,2023-07-23T00:42:11Z,1.50,5.25,4.00
3,Blackpool,Burton Albion,2023-08-05T14:00:00Z,Bet Victor,2023-07-23T00:41:48Z,1.55,4.80,4.00
4,Blackpool,Burton Albion,2023-08-05T14:00:00Z,Coral,2023-07-23T00:42:19Z,1.67,5.20,3.90
5,Blackpool,Burton Albion,2023-08-05T14:00:00Z,Ladbrokes,2023-07-23T00:42:02Z,1.67,5.20,3.90
6,Blackpool,Burton Albion,2023-08-05T14:00:00Z,888sport,2023-07-23T00:41:07Z,1.57,5.50,3.80
7,Blackpool,Burton Albion,2023-08-05T14:00:00Z,Betway,2023-07-23T00:41:28Z,1.55,4.75,3.80
8,Blackpool,Burton Albion,2023-08-05T14:00:00Z,Betfair,2023-07-23T00:40:35Z,1.55,4.90,4.10
9,Blackpool,Burton Albion,2023-08-05T14:00:00Z,Betfair,2023-07-23T00:40:35Z,1.76,6.20,5.80




Table for Bolton vs Lincoln City:


...

,Home Team,Away Team,date,Bookmaker Name,Last bookmaker update,team 1 win odds,team 2 win odds,Draw odds
0,Wycombe Wanderers,Exeter City,2023-08-05T14:00:00Z,BoyleSports,2023-07-23T00:42:27Z,1.83,4.00,3.20
1,Wycombe Wanderers,Exeter City,2023-08-05T14:00:00Z,Paddy Power,2023-07-23T00:41:25Z,1.91,3.70,3.40
2,Wycombe Wanderers,Exeter City,2023-08-05T14:00:00Z,Sky Bet,2023-07-23T00:42:11Z,1.85,3.75,3.40
3,Wycombe Wanderers,Exeter City,2023-08-05T14:00:00Z,Bet Victor,2023-07-23T00:41:48Z,1.87,3.60,3.50
4,Wycombe Wanderers,Exeter City,2023-08-05T14:00:00Z,Coral,2023-07-23T00:42:19Z,1.95,3.90,3.50
5,Wycombe Wanderers,Exeter City,2023-08-05T14:00:00Z,Ladbrokes,2023-07-23T00:42:02Z,1.91,3.90,3.50
6,Wycombe Wanderers,Exeter City,2023-08-05T14:00:00Z,888sport,2023-07-23T00:41:07Z,1.91,3.75,3.40
7,Wycombe Wanderers,Exeter City,2023-08-05T14:00:00Z,Betway,2023-07-23T00:41:28Z,1.85,3.50,3.40
8,Wycombe Wanderers,Exeter City,2023-08-05T14:00:00Z,Betfair,2023-07-23T00:40:35Z,1.88,3.90,3.55
9,Wycombe Wanderers,Exeter City,2023-08-05T14:00:00Z,Betfair,2023-07-23T00:40:35Z,2.10,4.70,4.20




Table for Northampton Town vs Stevenage:


,Home Team,Away Team,date,Bookmaker Name,Last bookmaker update,team 1 win odds,team 2 win odds,Draw odds
0,Northampton Town,Stevenage,2023-08-05T14:00:00Z,Betway,2023-07-23T00:41:28Z,2.60,2.50,2.88
1,Northampton Town,Stevenage,2023-08-05T14:00:00Z,Paddy Power,2023-07-23T00:41:25Z,2.62,2.60,3.10
2,Northampton Town,Stevenage,2023-08-05T14:00:00Z,888sport,2023-07-23T00:41:07Z,2.62,2.60,3.20
3,Northampton Town,Stevenage,2023-08-05T14:00:00Z,BoyleSports,2023-07-23T00:42:27Z,2.62,2.60,3.00
4,Northampton Town,Stevenage,2023-08-05T14:00:00Z,Ladbrokes,2023-07-23T00:42:02Z,2.75,2.75,3.00
5,Northampton Town,Stevenage,2023-08-05T14:00:00Z,Coral,2023-07-23T00:42:19Z,2.75,2.75,3.00
6,Northampton Town,Stevenage,2023-08-05T14:00:00Z,Sky Bet,2023-07-23T00:42:11Z,2.62,2.62,3.00
7,Northampton Town,Stevenage,2023-08-05T14:00:00Z,Betfair,2023-07-23T00:40:35Z,2.56,2.78,2.70
8,Northampton Town,Stevenage,2023-08-05T14:00:00Z,Betfair,2023-07-23T00:40:35Z,3.40,3.20,3.65
9,Northampton Town,Stevenage,2023-08-05T14:00:00Z,Virgin Bet,2023-07-23T00:42:23Z,2.55,2.55,2.88




Table for Reading vs Peterborough United:


,Home Team,Away Team,date,Bookmaker Name,Last bookmaker update,team 1 win odds,team 2 win odds,Draw odds
0,Reading,Peterborough United,2023-08-05T14:00:00Z,Betway,2023-07-23T00:41:28Z,2.70,2.40,3.00
1,Reading,Peterborough United,2023-08-05T14:00:00Z,Paddy Power,2023-07-23T00:41:25Z,2.62,2.40,3.30
2,Reading,Peterborough United,2023-08-05T14:00:00Z,888sport,2023-07-23T00:41:07Z,2.62,2.50,3.25
3,Reading,Peterborough United,2023-08-05T14:00:00Z,Bet Victor,2023-07-23T00:41:48Z,2.62,2.45,3.25
4,Reading,Peterborough United,2023-08-05T14:00:00Z,BoyleSports,2023-07-23T00:42:27Z,2.60,2.50,3.25
5,Reading,Peterborough United,2023-08-05T14:00:00Z,Ladbrokes,2023-07-23T00:42:02Z,2.87,2.45,3.25
6,Reading,Peterborough United,2023-08-05T14:00:00Z,Coral,2023-07-23T00:42:19Z,2.87,2.45,3.25
7,Reading,Peterborough United,2023-08-05T14:00:00Z,Sky Bet,2023-07-23T00:42:11Z,2.70,2.40,3.20
8,Reading,Peterborough United,2023-08-05T14:00:00Z,Betfair,2023-07-23T00:40:35Z,2.72,2.28,3.40
9,Reading,Peterborough United,2023-08-05T14:00:00Z,Betfair,2023-07-23T00:40:35Z,3.30,2.70,4.20


In [75]:
barnsley_vs_port_vale_df = match_tables.get("Barnsley vs Port Vale")

barnsley_vs_port_vale_df

,Home Team,Away Team,date,Bookmaker Name,Last bookmaker update,team 1 win odds,team 2 win odds,Draw odds
0,Barnsley,Port Vale,2023-08-05T14:00:00Z,BoyleSports,2023-07-23T00:42:27Z,1.70,4.75,3.30
1,Barnsley,Port Vale,2023-08-05T14:00:00Z,Paddy Power,2023-07-23T00:41:25Z,1.62,5.00,3.50
2,Barnsley,Port Vale,2023-08-05T14:00:00Z,Sky Bet,2023-07-23T00:42:11Z,1.70,4.50,3.50
3,Barnsley,Port Vale,2023-08-05T14:00:00Z,Bet Victor,2023-07-23T00:41:48Z,1.70,4.33,3.50
4,Barnsley,Port Vale,2023-08-05T14:00:00Z,Coral,2023-07-23T00:42:19Z,1.80,4.60,3.40
5,Barnsley,Port Vale,2023-08-05T14:00:00Z,Ladbrokes,2023-07-23T00:42:02Z,1.80,4.60,3.40
6,Barnsley,Port Vale,2023-08-05T14:00:00Z,888sport,2023-07-23T00:41:07Z,1.73,4.50,3.40
7,Barnsley,Port Vale,2023-08-05T14:00:00Z,Betway,2023-07-23T00:41:28Z,1.70,4.20,3.40
8,Barnsley,Port Vale,2023-08-05T14:00:00Z,Betfair,2023-07-23T00:40:35Z,1.68,4.40,3.20
9,Barnsley,Port Vale,2023-08-05T14:00:00Z,Betfair,2023-07-23T00:40:35Z,2.04,6.40,4.40


Now we perform some arbitrage. To do this, remember we want to find the lowest percentage, which means we want the maximum odds for each outcome.

In [96]:
# Calculate the maximum implied probabilities for each outcome
max_team1_prob = 1 / barnsley_vs_port_vale_df['team 1 win odds'].max()
max_team2_prob = 1 / barnsley_vs_port_vale_df['team 2 win odds'].max()
max_draw_prob = 1 / barnsley_vs_port_vale_df['Draw odds'].max()

total = max_team1_prob + max_team2_prob + max_draw_prob

if total < 1:
    print(f"Possible arbitrage opportunity detected with an edge of {(1 - total) * 100}%")

    # Find the bookmaker's name for each maximum odds value
    bookmaker_team1 = barnsley_vs_port_vale_df.loc[barnsley_vs_port_vale_df['team 1 win odds'].idxmax(), 'Bookmaker Name']
    bookmaker_team2 = barnsley_vs_port_vale_df.loc[barnsley_vs_port_vale_df['team 2 win odds'].idxmax(), 'Bookmaker Name']
    bookmaker_draw = barnsley_vs_port_vale_df.loc[barnsley_vs_port_vale_df['Draw odds'].idxmax(), 'Bookmaker Name']

    # Print the bookmaker's name for each outcome
    if bookmaker_team1:
        print(f"Use {bookmaker_team1} for Team 1 win")
    if bookmaker_team2:
        print(f"Use {bookmaker_team2} for Team 2 win")
    if bookmaker_draw:
        print(f"Use {bookmaker_draw} for Draw")
else:
    print("No arbitrage opportunity found.")

Possible arbitrage opportunity detected with an edge of 12.628119429590011%
Use Betfair for Team 1 win
Use Betfair for Team 2 win
Use Betfair for Draw
